In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import os

In [34]:
train_datagen = ImageDataGenerator(
    rescale=1./255,                  
    rotation_range=40,               
    width_shift_range=0.2,           
    height_shift_range=0.2,          
    shear_range=0.2,                 
    zoom_range=0.2,                  
    horizontal_flip=True,            
    fill_mode='nearest',             
)

In [35]:
train_generator = train_datagen.flow_from_directory(
    os.path.join("datasets","HAM10000_ESTRUTURADO","treino"),                        
    target_size=(500, 500),          
    batch_size=32,          
    class_mode='categorical'     
)

Found 648 images belonging to 4 classes.


In [36]:
val_generator = train_datagen.flow_from_directory(
    os.path.join("datasets","HAM10000_ESTRUTURADO","validacao"),                        
    target_size=(500, 500),          
    batch_size=32,          
    class_mode='categorical'     
)

Found 162 images belonging to 4 classes.


In [48]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(500, 500, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dropout(0.3),               
    Dense(4, activation='softmax')  
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','AUC'])

In [49]:
checkpoint = ModelCheckpoint("ham_multi_bestWeights_v3.h5",
                             monitor='val_accuracy',
                             verbose=1,
                             save_best_only=True,
                             mode='max',
                             save_weights_only=True)

In [47]:
model.load_weights("ham_multi_bestWeights_v3.h5")

ValueError: Layer count mismatch when loading weights from file. Model expected 6 layers, found 5 saved layers.

In [50]:
history = model.fit(
    train_generator, 
    steps_per_epoch=21,  
    epochs=4,            
    validation_data=val_generator,  
    validation_steps=6,
    callbacks = [checkpoint]
)

Epoch 1/4
21/21 [==============================] - ETA: 0s - loss: 3.4364 - accuracy: 0.2608 - auc: 0.4960
Epoch 1: val_accuracy improved from -inf to 0.25926, saving model to ham_multi_bestWeights_v3.h5
21/21 [==============================] - 80s 4s/step - loss: 3.4364 - accuracy: 0.2608 - auc: 0.4960 - val_loss: 1.3818 - val_accuracy: 0.2593 - val_auc: 0.5504
Epoch 2/4
21/21 [==============================] - ETA: 0s - loss: 1.3769 - accuracy: 0.3009 - auc: 0.5504
Epoch 2: val_accuracy improved from 0.25926 to 0.29012, saving model to ham_multi_bestWeights_v3.h5
21/21 [==============================] - 77s 4s/step - loss: 1.3769 - accuracy: 0.3009 - auc: 0.5504 - val_loss: 1.3933 - val_accuracy: 0.2901 - val_auc: 0.5600
Epoch 3/4
21/21 [==============================] - ETA: 0s - loss: 1.4139 - accuracy: 0.2608 - auc: 0.5288
Epoch 3: val_accuracy did not improve from 0.29012
21/21 [==============================] - 75s 4s/step - loss: 1.4139 - accuracy: 0.2608 - auc: 0.5288 - val_lo